In [ ]:
import random
import numpy as np

In [ ]:
#using sigmoid and cross-entropy
class creat_network(object):
    
    def __init__(self,len_x,len_h1,len_h2,len_y):    #define number of neurons in different layers
        self.len_x = len_x
        self.len_h1 = len_h1
        self.len_h2 = len_h2
        self.len_y = len_y
        self.biases = [np.random.randn(i,1) for i in [len_h1,len_h2,len_y]]
        self.weights = [np.random.randn(i,j) for i,j in zip([len_h1,len_h2,len_y],[len_x,len_h1,len_h2])]
    


        
    def forward_pass(self,a):   #for 1st layer a=x(input)
        for w,b in zip(self.weights,self.biases):
            a = sigmoid(np.dot(w,a)+b)
        return a
    
    
    #len_m_batch: lenght of mini batch for SGD
    #alpha: learning rate
    #reg: l2 regularization parameter
    
    def sgd(self,train_data,test_data, epochs, len_m_batch, alpha, reg):   
        len_test = len(test_data)
        len_train = len(train_data)
        for _ in xrange(epochs):
            random.shuffle(train_data)
            mini_batches = [train_data[i:len_m_batch+i] for i in xrange(0,len_train,len_m_batch)] 
            
            for batch in mini_batches:
                w_after_iteration = [np.zeros(w.shape) for w in self.weights]
                b_after_iteration = [np.zeros(b.shape) for b in self.biases]
                for x,y in batch:
                    delta_w,delta_b = self.backpropogate(x,y)
                    w_after_iteration = [w+dw for w,dw in zip(w_after_iteration,delta_w)]
                    b_after_iteration = [b+db for b,db in zip(b_after_iteration,delta_b)]
                    
                self.weights = [((1-alpha*(reg/len_train))*w-((alpha*1.0)/len_m_batch)*dw) for w,dw in zip(self.weights,w_after_iteration)] #using l2 regularization
                self.biases = [(b-((alpha*1.0)/len_m_batch)*db) for b,db in zip(self.biases,b_after_iteration)]
                
            print("Epoch {0}: \nTest Accuracy: {1}/{2}\n\n".format(_,self.evaluate_test(test_data),len_test))
                    
                    
                    
    def backpropogate(self,x,y):
        backpass_w = [np.zeros(w.shape) for w in self.weights]
        backpass_b = [np.zeros(b.shape) for b in self.biases]
        a = x
        a_list = [a]
        z_list = [] 
        for w,b in zip(self.weights,self.biases):
            z = np.dot(w,a)+b
            z_list.append(z)
            a = sigmoid(z)
            a_list.append(a)
            
        delta = derivative_cost(a_list[-1],y)
        backpass_b[-1] = delta
        backpass_w[-1] = np.dot(delta,(a_list[-2].transpose()))
        
        for l in range(2,4):
            z = z_list[-l]
            sig_derivative = sigmoid(z)*(1-sigmoid(z))
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sig_derivative
            backpass_b[-l] = delta
            backpass_w[-l] = np.dot(delta, a_list[-l-1].transpose())
            
        return (backpass_w,backpass_b)
    
    def evaluate_test(self, data):
        results = [(np.argmax(self.forward_pass(x)), y) for (x, y) in data]
        return sum(int(x == y) for (x, y) in results)
        
    def evaluate_train(self,data):
        results = [(np.argmax(self.forward_pass(x)), np.agrmax(y)) for (x, y) in data]
        #print results
        return sum(int(x == y) for (x, y) in results)
        
def cost_fun(a,y):
    return -np.sum(((1-y)*np.log(1-a))+((y)*np.log(a))) #cross-entropy cost func.
    
        
def derivative_cost(a,y):
    return a-y


def shuffle(data):
    np.random.seed(0)
    np.random.shuffle(data)
    return data

def sigmoid(f):
    return 1.0/(1+np.exp(-f))
        

In [ ]:
#to use:
#net = creat_network(x, lenth_h1, length_h2, classes)
#net.sgd(training_data,test_data,epochs,lenght_mini_batch,learning_rate,regularization_parameter)

#vectorize(one hot encode) target_classes for training data